In [ ]:
# Import basic packages 
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 10 # Ensure reproducible results
%matplotlib inline

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#import all data
data = pd.read_csv(r'/content/drive/MyDrive/msc-project-data/features_with_outcome.csv',encoding='utf=8',index_col='org_uuid')

y_data = data['outcome']
X_data = data.drop(columns = ['outcome'])

#splitting data into test and full training set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, test_size = 0.2,stratify=y_data)

Mounted at /content/drive


In [ ]:
len(data)

28727

In [ ]:
X_data.head(3)

,people_num_founder,people_is_founder_gender_diversity,people_num_bachelor,people_num_master,people_num_phd,people_num_mba,people_num_stem,people_num_ivy_league,people_num_top_100,people_num_advisor,people_founder_experience_num_years,people_founder_experience_num_companies,people_founder_experience_num_successful,people_founder_experience_num_same_category,ecosystem_is_administrative_services,ecosystem_is_advertising,ecosystem_is_agriculture_and_farming,ecosystem_is_apps,ecosystem_is_artificial_intelligence,ecosystem_is_biotechnology,ecosystem_is_clothing_and_apparel,ecosystem_is_commerce_and_shopping,ecosystem_is_community_and_lifestyle,ecosystem_is_consumer_electronics,ecosystem_is_consumer_goods,ecosystem_is_content_and_publishing,ecosystem_is_data_and_analytics,ecosystem_is_design,ecosystem_is_education,ecosystem_is_energy,ecosystem_is_events,ecosystem_is_financial_services,ecosystem_is_food_and_beverage,ecosystem_is_gaming,ecosystem_is_government_and_military,ecosystem_is_hardware,ecosystem_is_health_care,ecosystem_is_information_technology,ecosystem_is_internet_services,ecosystem_is_lending_and_investments,...,ecosystem_is_navigation_and_mapping,ecosystem_is_other,ecosystem_is_payments,ecosystem_is_platforms,ecosystem_is_privacy_and_security,ecosystem_is_professional_services,ecosystem_is_real_estate,ecosystem_is_sales_and_marketing,ecosystem_is_science_and_engineering,ecosystem_is_software,ecosystem_is_sports,ecosystem_is_sustainability,ecosystem_is_transportation,ecosystem_is_travel_and_tourism,ecosystem_is_video,ecosystem_is_country_can,ecosystem_is_country_chn,ecosystem_is_country_deu,ecosystem_is_country_esp,ecosystem_is_country_fra,ecosystem_is_country_gbr,ecosystem_is_country_ind,ecosystem_is_country_isr,ecosystem_is_country_rus,ecosystem_is_country_usa,ecosystem_is_country_others,ecosystem_kauffman_rate_of_new_entrepreneur,ecosystem_kauffman_opportunity_share,ecosystem_kauffman_startup_early_job_creation,ecosystem_kauffman_startup_early_survival_rate,ecosystem_kauffman_zindex,network_has_facebook_url,network_has_linkedin_url,network_has_twitter_url,network_has_homepage,network_is_attend_top_event,network_has_description,investor_is_cbtop1000,investor_is_no_rank,investor_is_not_cbtop1000
org_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9dc17185-743d-5fbc-f03b-ad0c81c8795d,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.3,3,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00,0,0,0,0,0,1,0,1,0
9c8adac6-5c8e-9344-b763-6beab966c63c,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.16,80.54,7.39,82.46,-0.94,0,0,0,1,0,1,0,1,0
6749cc07-efed-ff09-4efe-43faf6f580de,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00,0,0,0,1,0,1,0,0,1


In [ ]:
y_train.value_counts()

0    17652
1     5329
Name: outcome, dtype: int64

In [ ]:
y_test.value_counts()

0    4414
1    1332
Name: outcome, dtype: int64

In [ ]:
def compute_score (y_act,y_pred,y_pred_proba):
    
    from sklearn.metrics import accuracy_score,log_loss,recall_score,precision_score,f1_score
    
    accuracy = accuracy_score (y_act,y_pred)
    log_loss = log_loss (y_act,y_pred_proba)
    recall = recall_score (y_act,y_pred)
    precision = precision_score (y_act,y_pred)
    f1 = f1_score (y_act,y_pred)
    
    return accuracy,log_loss,recall,precision,f1

In [ ]:
#run classifier

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
random_state = 10

names = ["Dummy, most frequent", 
         "Gaussian Naive Bayes", 
         "Random Forest", 
         "Xgboost", 
         "Linear SVM", 
         "RBF SVM", 
         "MLP", 
         "MLP stronger reg", 
         ]

classifiers = [
    DummyClassifier(strategy='most_frequent'),
    GaussianNB(),
    RandomForestClassifier(max_depth=10, n_estimators=50,random_state=random_state),
    XGBClassifier(),
    SVC(kernel="linear", probability=True, random_state=random_state),
    SVC(kernel='rbf', probability=True, random_state=random_state),
    MLPClassifier(random_state=random_state, max_iter=800),  # default regularisation
    MLPClassifier(random_state=random_state, max_iter=800, alpha=1),  # more regularisation
    ]

# Training score
ca_train_score = {}  # Classification accuracy
ce_train_score = {}  # Cross-entropy
re_train_score = {}  # recall
pr_train_score = {}  # precision
f1_train_score = {}  # f1 score

# Tests score
ca_test_score = {} 
ce_test_score = {}
re_test_score = {}
pr_test_score = {}
f1_test_score = {}

for name, clf in zip(names, classifiers):
    print('Running for {}'.format(name))
    clf.fit(X_train, y_train)
    
    y_train_predict = clf.predict(X_train)
    y_train_predict_proba = clf.predict_proba(X_train)
    acc, log_loss, re, pr, f1 = compute_score (y_train,y_train_predict,y_train_predict_proba)
    
    ca_train_score[name] = acc
    ce_train_score[name] = log_loss
    re_train_score[name] = re
    pr_train_score[name] = pr
    f1_train_score[name] = f1
    
    y_test_predict = clf.predict(X_test)
    y_test_predict_proba = clf.predict_proba(X_test)
    acc, log_loss, re, pr, f1 = compute_score (y_test,y_test_predict,y_test_predict_proba)
    ca_test_score[name] = acc
    ce_test_score[name] = log_loss
    re_test_score[name] = re
    pr_test_score[name] = pr
    f1_test_score[name] = f1

Running for Dummy, most frequent
Running for Gaussian Naive Bayes


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running for Random Forest
Running for Xgboost
Running for Linear SVM
Running for RBF SVM
Running for MLP
Running for MLP stronger reg


In [ ]:
#Print Result
print('Classification performance on test set: \n')
print("{0:<20s}   {1:-^62s}   {2:-^62s}".format('','Test', 'Training'))
print("{0:<20s}{1:>10s}{2:>10s}{3:>10s}{4:>10s}{5:>10s}{6:>13s}{7:>13s}{8:>10s}{9:>10s}{10:>10s}".format(
    'Method', 
    'Accuracy\u2191', 'Log Loss\u2193', 'Precision\u2191','Recall\u2191','F1\u2191',
    'Accuracy\u2191', 'Log Loss\u2193', 'Precision\u2191','Recall\u2191','F1\u2191'))
print("-"*(20+10*13))
for clf in names:
    print ("{method:<20s}{test_accuracy:>13.3f}{test_log_loss:>13.3f}{test_precision:>13.3f}{test_recall:>13.3f}{test_f1:>13.3f}{train_accuracy:>13.3f}{train_log_loss:>13.3f}{train_precision:>13.3f}{train_recall:>13.3f}{train_f1:>13.3f}".format(
        method=clf, 
        test_accuracy=ca_test_score[clf], test_log_loss=ce_test_score[clf], test_precision=pr_test_score[clf], test_recall=re_test_score[clf], test_f1=f1_test_score[clf],
        train_accuracy=ca_train_score[clf], train_log_loss=ce_train_score[clf], train_precision=pr_train_score[clf], train_recall=re_train_score[clf], train_f1=f1_train_score[clf]))

Classification performance on test set: 

                       -----------------------------Test-----------------------------   ---------------------------Training---------------------------
Method                  Accuracy↑    Log Loss↓   Precision↑      Recall↑          F1↑    Accuracy↑    Log Loss↓   Precision↑      Recall↑          F1↑
------------------------------------------------------------------------------------------------------------------------------------------------------
Dummy, most frequent        0.768        8.007        0.000        0.000        0.000        0.768        8.009        0.000        0.000        0.000
Gaussian Naive Bayes        0.718        3.157        0.402        0.447        0.423        0.726        2.973        0.420        0.480        0.448
Random Forest               0.785        0.480        0.655        0.151        0.245        0.818        0.432        0.940        0.229        0.368
Xgboost                     0.786        0.472      